[View in Colaboratory](https://colab.research.google.com/github/RayleighKim/M10_Intro_to_Classification/blob/master/10_1_Churn_Prediction.ipynb)

# Intro to Classification 01

## Churn Prediction!

### Your name:

#### 실습목표<br>
1. Scikit-Learn 을 이용하여 Logistic Regression을 할 줄 안다.
2. Scikit-Learn 을 이용하여 Artificial Neural Network를 사용할 줄 안다.
---------------
Rayleigh Kim @ D:plus

In [0]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

## Data Loading & Preparation

이번 실습은 다음의 정말 중요한 과정이 생략될 것이다.

Raw_data 에서, **어떤 과정**을 거치면 굉장히 잘 정돈된 Data를 얻게 된다.

**어떤 과정이란 다음과 같다.**
1. 원래 데이터는 어떻게 생겼는가?
2. 문제 정의
    * 이탈자는 어떻게 정의할 것인가.
    * 어떤 기간의 데이터로, 어떤 기간을 예측할 것인가.
3. Data Wrangling & Feature Engineering


다음의 데이터들을 구경해보자. 로컬에서 csv파일을 엑셀로 열어도 좋다.

동시에, **(new주석)RetailChurnTemplate_FeatureEngg.xlsx** 파일을 열어서 감상하자.


    

In [4]:
'''
고객 정보 데이터
'''
user_info = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurn_UserInfoData.csv')
user_info.head()

,UserId,Age,Address,Gender,UserType
0,2105345,D,F,Unknown,Unknown
1,2085920,F,F,Unknown,Unknown
2,1976717,C,C,Unknown,Unknown
3,768535,A,E,Unknown,Unknown
4,1895117,D,F,Unknown,Unknown


In [5]:
'''
Transaction 데이터
'''

raw_data = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurn_Transaction.csv')
raw_data.head()

,UserId,TransactionId,Timestamp,ItemId,Quantity,Value,Location,ProductCategory
0,2105345,1215553,12/31/2000 12:00:00 AM,4.710040e+12,1,149,Unknown,Unknown
1,2105345,1216545,12/31/2000 12:00:00 AM,4.711090e+12,1,179,Unknown,Unknown
2,2105345,1216590,12/31/2000 12:00:00 AM,9.556000e+12,1,28,Unknown,Unknown
3,2105345,1217249,12/31/2000 12:00:00 AM,4.711800e+12,1,199,Unknown,Unknown
4,2105345,1217259,12/31/2000 12:00:00 AM,4.710030e+12,1,139,Unknown,Unknown


하지만 우리는 바로 아래의 데이터를 사용할 것이다.

In [53]:
churn = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurnTemplate_FeatureEngg.csv')
'''
아래의 코드는 row를 셔플하기 위해 필요하다.
'''
churn = churn.sample(frac = 1, random_state = 2018).reset_index(drop=True)

churn.head()

,UserId,Age,Address,Gender,UserType,Label,Total_Quantity,Total_Value,StDev_Quantity,StDev_Value,...,Unique_Location,Unique_ProductCategory,Total_Quantity_per_Unique_TransactionId,Total_Quantity_per_Unique_ItemId,Total_Quantity_per_Unique_Location,Total_Quantity_per_Unique_ProductCategory,Total_Value_per_Unique_TransactionId,Total_Value_per_Unique_ItemId,Total_Value_per_Unique_Location,Total_Value_per_Unique_ProductCategory
0,1781991,E,E,Unknown,Unknown,Nonchurner,49,4844,0.350605,100.809789,...,1,1,1.113636,1.484848,24.5,24.5,110.090909,146.787879,2422.0,2422.0
1,412001,D,A,Unknown,Unknown,Churner,46,3940,0.426675,64.288612,...,1,1,1.121951,1.642857,23.0,23.0,96.097561,140.714286,1970.0,1970.0
2,2055312,E,C,Unknown,Unknown,Churner,42,2715,1.411149,125.075847,...,1,1,1.826087,2.210526,21.0,21.0,118.043478,142.894737,1357.5,1357.5
3,2090665,C,F,Unknown,Unknown,Churner,5,3075,0.500000,555.658393,...,1,1,1.000000,1.666667,2.5,2.5,615.000000,1025.000000,1537.5,1537.5
4,1584844,E,C,Unknown,Unknown,Churner,19,1206,2.751623,149.295807,...,1,1,2.375000,2.375000,9.5,9.5,150.750000,150.750000,603.0,603.0


In [54]:
# User는 따로 떼어 둬야 어떤 User인지 알 수 있다.
Users = churn[['UserId']]

# Unknown 투성이인 Gender 와 UserType을 버리자
churn_dropped = churn.drop(['UserId', 'Gender', 'UserType'], axis=1)
churn_dropped.head()

,Age,Address,Label,Total_Quantity,Total_Value,StDev_Quantity,StDev_Value,AvgTimeDelta,Recency,Unique_TransactionId,...,Unique_Location,Unique_ProductCategory,Total_Quantity_per_Unique_TransactionId,Total_Quantity_per_Unique_ItemId,Total_Quantity_per_Unique_Location,Total_Quantity_per_Unique_ProductCategory,Total_Value_per_Unique_TransactionId,Total_Value_per_Unique_ItemId,Total_Value_per_Unique_Location,Total_Value_per_Unique_ProductCategory
0,E,E,Nonchurner,49,4844,0.350605,100.809789,1.697674,15,43,...,1,1,1.113636,1.484848,24.5,24.5,110.090909,146.787879,2422.0,2422.0
1,D,A,Churner,46,3940,0.426675,64.288612,1.225000,3,40,...,1,1,1.121951,1.642857,23.0,23.0,96.097561,140.714286,1970.0,1970.0
2,E,C,Churner,42,2715,1.411149,125.075847,0.954545,17,22,...,1,1,1.826087,2.210526,21.0,21.0,118.043478,142.894737,1357.5,1357.5
3,C,F,Churner,5,3075,0.500000,555.658393,0.000000,67,4,...,1,1,1.000000,1.666667,2.5,2.5,615.000000,1025.000000,1537.5,1537.5
4,E,C,Churner,19,1206,2.751623,149.295807,0.142857,37,7,...,1,1,2.375000,2.375000,9.5,9.5,150.750000,150.750000,603.0,603.0


In [55]:
# 어딘가에 있을지 모를 NA, NaN 등을 찾자. 
churn_dropped.isna().any()

Age                                          False
Address                                      False
Label                                        False
Total_Quantity                               False
Total_Value                                  False
StDev_Quantity                               False
StDev_Value                                  False
AvgTimeDelta                                 False
Recency                                      False
Unique_TransactionId                         False
Unique_ItemId                                False
Unique_Location                              False
Unique_ProductCategory                       False
Total_Quantity_per_Unique_TransactionId      False
Total_Quantity_per_Unique_ItemId             False
Total_Quantity_per_Unique_Location           False
Total_Quantity_per_Unique_ProductCategory    False
Total_Value_per_Unique_TransactionId         False
Total_Value_per_Unique_ItemId                False
Total_Value_per_Unique_Location

더미를 이용하여 펼치자.

컴퓨터는 A, B, C, D라고 입력되어 있어봤자 그걸 그대로 인식할 수 없다!

In [56]:
churn_dropped.head()

,Age,Address,Label,Total_Quantity,Total_Value,StDev_Quantity,StDev_Value,AvgTimeDelta,Recency,Unique_TransactionId,...,Unique_Location,Unique_ProductCategory,Total_Quantity_per_Unique_TransactionId,Total_Quantity_per_Unique_ItemId,Total_Quantity_per_Unique_Location,Total_Quantity_per_Unique_ProductCategory,Total_Value_per_Unique_TransactionId,Total_Value_per_Unique_ItemId,Total_Value_per_Unique_Location,Total_Value_per_Unique_ProductCategory
0,E,E,Nonchurner,49,4844,0.350605,100.809789,1.697674,15,43,...,1,1,1.113636,1.484848,24.5,24.5,110.090909,146.787879,2422.0,2422.0
1,D,A,Churner,46,3940,0.426675,64.288612,1.225000,3,40,...,1,1,1.121951,1.642857,23.0,23.0,96.097561,140.714286,1970.0,1970.0
2,E,C,Churner,42,2715,1.411149,125.075847,0.954545,17,22,...,1,1,1.826087,2.210526,21.0,21.0,118.043478,142.894737,1357.5,1357.5
3,C,F,Churner,5,3075,0.500000,555.658393,0.000000,67,4,...,1,1,1.000000,1.666667,2.5,2.5,615.000000,1025.000000,1537.5,1537.5
4,E,C,Churner,19,1206,2.751623,149.295807,0.142857,37,7,...,1,1,2.375000,2.375000,9.5,9.5,150.750000,150.750000,603.0,603.0


In [57]:
pd.get_dummies(churn_dropped[['Age','Address']]).head()

,Age_A,Age_B,Age_C,Age_D,Age_E,Age_F,Age_G,Age_H,Age_I,Age_J,Age_K,Address_A,Address_B,Address_C,Address_D,Address_E,Address_F,Address_G,Address_H
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [58]:
churn_processed = pd.concat([churn_dropped, pd.get_dummies(churn_dropped[['Age','Address']])], axis=1).drop(['Age','Address'], axis=1)

churn_processed['label'] = 1
churn_processed.loc[churn_processed['Label']=='Nonchurner', 'label'] = 0

churn_ready = churn_processed.drop('Label', axis=1)

churn_ready.head()

,Total_Quantity,Total_Value,StDev_Quantity,StDev_Value,AvgTimeDelta,Recency,Unique_TransactionId,Unique_ItemId,Unique_Location,Unique_ProductCategory,...,Age_K,Address_A,Address_B,Address_C,Address_D,Address_E,Address_F,Address_G,Address_H,label
0,49,4844,0.350605,100.809789,1.697674,15,43,32,1,1,...,0,0,0,0,0,1,0,0,0,0
1,46,3940,0.426675,64.288612,1.225000,3,40,27,1,1,...,0,1,0,0,0,0,0,0,0,1
2,42,2715,1.411149,125.075847,0.954545,17,22,18,1,1,...,0,0,0,1,0,0,0,0,0,1
3,5,3075,0.500000,555.658393,0.000000,67,4,2,1,1,...,0,0,0,0,0,0,1,0,0,1
4,19,1206,2.751623,149.295807,0.142857,37,7,7,1,1,...,0,0,0,1,0,0,0,0,0,1


## Train set, Test set

시계열은 아니다!

데이터의 모양을 먼저 본다

In [59]:
churn_processed.shape

(9159, 39)

In [0]:
X_train, X_test, Y_train, Y_test = churn_ready.loc[:7000].drop('label', axis=1), \
churn_ready.loc[7001:].drop('label', axis=1),\
churn_ready.loc[:7000, 'label'],\
churn_ready.loc[7001:, 'label']

In [61]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(7001, 37) (2158, 37) (7001,) (2158,)


## Logistic Regression

In [62]:
clf = LogisticRegression()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}%'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}%'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))
pd.DataFrame({'colnames':X_train.columns, 'coef':clf.coef_.reshape(-1,)})

트레이닝 7001명 중 실제 churner의 비율 :  0.6011998285959149
트레이닝 7001명에 대해서 예측된 churner의 비율 : 0.7253249535780603
모델의 트레이닝셋위의 정확도는 : 69.33%
--------------------------------------------
테스트 2158명 중 실제 churner의 비율 :  0.6019462465245597
테스트 2158명에 대해서 예측된 churner의 비율 : 0.7242817423540315
모델의 테스트셋위의 정확도는 : 69.14%
---------------------------------------------
[[ 394  201]
 [ 465 1098]]
---------------------------------------------
precision : True라고 예측한 것 중 진짜 True 비율
recall : 진짜 True 중 True라고 예측된 것의 비율
             precision    recall  f1-score   support

Non-churner       0.66      0.46      0.54       859
    churner       0.70      0.85      0.77      1299

avg / total       0.69      0.69      0.68      2158



,coef,colnames
0,0.000004,Total_Quantity
1,0.000017,Total_Value
2,0.031580,StDev_Quantity
3,-0.000306,StDev_Value
4,-0.083909,AvgTimeDelta
5,0.009119,Recency
6,-0.034433,Unique_TransactionId
7,0.007460,Unique_ItemId
8,0.332682,Unique_Location
9,0.332682,Unique_ProductCategory


## K- nearest Neighbor  Classification

In [63]:
# 아래의 n_neighbors를 바꿔가며 관찰해보자.

clf = KNeighborsClassifier(n_neighbors = 30)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))


트레이닝 7001명 중 실제 churner의 비율 :  0.6011998285959149
트레이닝 7001명에 대해서 예측된 churner의 비율 : 0.6899014426510498
모델의 트레이닝셋위의 정확도는 : 68.39
--------------------------------------------
테스트 2158명 중 실제 churner의 비율 :  0.6019462465245597
테스트 2158명에 대해서 예측된 churner의 비율 : 0.6913809082483782
모델의 테스트셋위의 정확도는 : 65.57
---------------------------------------------
[[ 391  275]
 [ 468 1024]]
---------------------------------------------
precision : True라고 예측한 것 중 진짜 True 비율
recall : 진짜 True 중 True라고 예측된 것의 비율
             precision    recall  f1-score   support

Non-churner       0.59      0.46      0.51       859
    churner       0.69      0.79      0.73      1299

avg / total       0.65      0.66      0.65      2158



## Random Forest

[공식 사이킷 런 문서](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) 를 참고하여 튜닝해보자.

반드시 알아야 할 것은 다음과 같다.

* n_estimators
* max_depth
* min_samples_split
* min_samples_leaf
* max_leaf_nodes


In [64]:
# 아래의 n_neighbors를 바꿔가며 관찰해보자.

clf = RandomForestClassifier(n_estimators = 30,
                            max_depth = None,
                            min_samples_split = 5,
                            min_samples_leaf = 1,
                            max_leaf_nodes = None)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))


트레이닝 7001명 중 실제 churner의 비율 :  0.6011998285959149
트레이닝 7001명에 대해서 예측된 churner의 비율 : 0.6120554206541923
모델의 트레이닝셋위의 정확도는 : 98.29
--------------------------------------------
테스트 2158명 중 실제 churner의 비율 :  0.6019462465245597
테스트 2158명에 대해서 예측된 churner의 비율 : 0.6733086190917517
모델의 테스트셋위의 정확도는 : 68.58
---------------------------------------------
[[ 443  262]
 [ 416 1037]]
---------------------------------------------
precision : True라고 예측한 것 중 진짜 True 비율
recall : 진짜 True 중 True라고 예측된 것의 비율
             precision    recall  f1-score   support

Non-churner       0.63      0.52      0.57       859
    churner       0.71      0.80      0.75      1299

avg / total       0.68      0.69      0.68      2158

